In [45]:
%pip install numpy pandas scikit-learn tensorflow

Note: you may need to restart the kernel to use updated packages.


In [46]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [47]:
df = pd.read_csv("/kaggle/input/dataset/Cleaned_Indian_Food_Dataset.csv")

In [48]:
# Preprocessing function (optional but recommended)
def preprocess_text(text):
    text = str(text).lower().strip()       # Ensure string, lowercase, and strip
    text = ' '.join(text.split())          # Remove extra whitespace
    return text

In [49]:
# Apply preprocessing
input_texts = [preprocess_text(text) for text in df["Cleaned-Ingredients"]]
target_texts = ["<start> " + preprocess_text(text) + " <end>" for text in df["TranslatedInstructions"]]

In [50]:
encoder_tokenizer = Tokenizer()
encoder_tokenizer.fit_on_texts(input_texts)
encoder_sequences = encoder_tokenizer.texts_to_sequences(input_texts)
encoder_input_data = pad_sequences(encoder_sequences, padding='post')

In [51]:
decoder_tokenizer = Tokenizer(filters='')
decoder_tokenizer.fit_on_texts(target_texts)
reverse_decoder_word_index = {index: word for word, index in decoder_tokenizer.word_index.items()}
decoder_sequences = decoder_tokenizer.texts_to_sequences(target_texts)
decoder_input_data = pad_sequences([seq[:-1] for seq in decoder_sequences], padding='post')
decoder_target_data = pad_sequences([seq[1:] for seq in decoder_sequences], padding='post')

In [52]:
import pickle

# Load encoder tokenizer
with open('/kaggle/input/final-food-rep/encoder_tokenizer.pkl', 'rb') as f:
    encoder_tokenizer = pickle.load(f)

# Load decoder tokenizer
with open('/kaggle/input/final-food-rep/decoder_tokenizer.pkl', 'rb') as f:
    decoder_tokenizer = pickle.load(f)

In [53]:
encoder_vocab_size = len(encoder_tokenizer.word_index) + 1
decoder_vocab_size = len(decoder_tokenizer.word_index) + 1
embedding_dim = 128
latent_dim = 256

In [54]:
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(encoder_vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

In [55]:
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(decoder_vocab_size, embedding_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(decoder_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [56]:
# model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# model.summary()

In [57]:
# from keras.callbacks import ModelCheckpoint

# checkpoint = ModelCheckpoint(
#     filepath='/kaggle/working/model_check.weights.h5',  # or .h5 or .weights.h5
#     save_weights_only=True,
#     save_best_only=False,
#     save_freq='epoch',
#     verbose=1
# )

In [58]:
# Remove all extra dimensions
decoder_target_data = np.squeeze(decoder_target_data)

# Add just one last dimension
decoder_target_data = np.expand_dims(decoder_target_data, -1)

# Confirm final shape
print(decoder_target_data.shape)  

(5938, 1005, 1)


In [59]:
# from keras.callbacks import EarlyStopping

# early_stopping = EarlyStopping(
#     monitor='val_loss',
#     patience=3,  # stops if val_loss doesn't improve for 3 consecutive epochs
#     restore_best_weights=True
# )

# model.fit(
#     [encoder_input_data, decoder_input_data],
#     decoder_target_data,
#     batch_size=16,
#     epochs=50,
#     validation_split=0.2,
#     callbacks=[early_stopping]
# )

In [60]:

from tensorflow.keras.models import load_model
# model.save('/kaggle/working/seq2seq_model.h5')
model= load_model("/kaggle/input/final-food-rep/seq2seq_model.h5")

In [61]:
# Encoder inference model
encoder_model = Model(encoder_inputs, encoder_states)

In [62]:
# Decoder inference model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = dec_emb_layer(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2
)

In [63]:
def decode_sequence(input_seq):
    # Encode the input sequence
    states_value = encoder_model.predict(input_seq)

    # Initialize the target sequence with the <start> token
    target_seq = np.array([[decoder_tokenizer.word_index['<start>']]])


    stop_condition = False
    decoded_tokens = []
    max_target_len = 100  # You can increase this

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the token with the highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = decoder_tokenizer.index_word.get(sampled_token_index, '')

        # Stop if <end> token or max length is reached
        if sampled_token == '<end>' or len(decoded_tokens) > max_target_len:
            stop_condition = True
        else:
            if len(decoded_tokens) == 0 or sampled_token != decoded_tokens[-1]:  # Prevent repetition
                decoded_tokens.append(sampled_token)

            # Update target sequence and states
            target_seq = np.array([[sampled_token_index]])
            states_value = [h, c]
            

    return ' '.join(decoded_tokens)

In [64]:
import pickle
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
import ipywidgets as widgets
from IPython.display import display, clear_output  # Only fatal errors


# Load model
model = load_model('/kaggle/input/final-food-rep/seq2seq_model.h5')

# Load tokenizers
with open('/kaggle/input/final-food-rep/encoder_tokenizer.pkl', 'rb') as f:
    encoder_tokenizer = pickle.load(f)
with open('/kaggle/input/final-food-rep/decoder_tokenizer.pkl', 'rb') as f:
    decoder_tokenizer = pickle.load(f)

reverse_decoder_word_index = {i: word for word, i in decoder_tokenizer.word_index.items()}
start_token_index = decoder_tokenizer.word_index['start']
end_token_index = decoder_tokenizer.word_index['end']

max_encoder_seq_length = model.input[0].shape[1]
max_decoder_seq_length = model.output.shape[1]

# --- Decoding function (Greedy or Beam) ---
def decode_sequence(input_seq):
    # Encode the input sequence
    states_value = encoder_model.predict(input_seq)

    # Initialize the target sequence with the <start> token
    target_seq = np.array([[decoder_tokenizer.word_index['<start>']]])


    stop_condition = False
    decoded_tokens = []
    max_target_len = 100  # You can increase this

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Get the token with the highest probability
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = decoder_tokenizer.index_word.get(sampled_token_index, '')

        # Stop if <end> token or max length is reached
        if sampled_token == '<end>' or len(decoded_tokens) > max_target_len:
            stop_condition = True
        else:
            if len(decoded_tokens) == 0 or sampled_token != decoded_tokens[-1]:  # Prevent repetition
                decoded_tokens.append(sampled_token)

            # Update target sequence and states
            target_seq = np.array([[sampled_token_index]])
            states_value = [h, c]
            

    return ' '.join(decoded_tokens)
    
# --- UI Components ---
input_box = widgets.Text(
    value='onion tomato garlic',
    placeholder='Enter ingredients...',
    description='Ingredients:',
    layout=widgets.Layout(width='600px')
)
output_box = widgets.Output()
button = widgets.Button(description="Generate Instruction", button_style='success')

def on_button_click(b):
    output_box.clear_output()
    input_text = input_box.value.lower()
    sequence = encoder_tokenizer.texts_to_sequences([input_text])
    padded_seq = pad_sequences(sequence, maxlen=max_encoder_seq_length, padding='post')
    
    with output_box:
        print("Model loaded. Now predicting...")
        prediction = decode_sequence(padded_seq)
        print(f"Instruction:\n{prediction}")

button.on_click(on_button_click)

# --- Display UI ---
display(input_box, button, output_box)


Text(value='onion tomato garlic', description='Ingredients:', layout=Layout(width='600px'), placeholder='Enter…

Button(button_style='success', description='Generate Instruction', style=ButtonStyle())

Output()

In [67]:
def evaluate_recipe_quality(generated_text):
    metrics = {}
    
    # 1. Coherence Score (0-1)
    metrics['coherence'] = check_sentence_structure(generated_text)
    
    # 2. Repetition Score (lower is better)
    metrics['repetition'] = calculate_repetition_penalty(generated_text)
    
    # 3. Recipe Completeness (0-1)
    metrics['completeness'] = check_recipe_steps(generated_text)
    
    # 4. Indian Context Score (0-1)
    metrics['indian_context'] = check_indian_cooking_terms(generated_text)
    
    return metrics

In [65]:
# import re
# import numpy as np
# from collections import defaultdict

# # Common Indian cooking verbs and connector words to guide generation
# INDIAN_COOKING_VERBS = [
#     "cook", "fry", "roast", "toast", "boil", "simmer", "sauté", "temper", "garnish", "grind", 
#     "blend", "mix", "knead", "ferment", "steam", "pressure cook", "stir", "add", "sprinkle", 
#     "pour", "marinate", "soak", "drain", "strain", "chop", "dice", "mince", "grate", "serve"
# ]

# CONNECTOR_WORDS = [
#     "then", "next", "after", "finally", "meanwhile", "until", "when", "and", "first", "lastly",
#     "once", "allow", "let", "keep", "continue", "before", "while"
# ]

# # Common Indian spices and ingredients to boost in vocabulary
# INDIAN_INGREDIENTS = [
#     "cumin", "coriander", "turmeric", "cardamom", "cloves", "cinnamon", "mustard seeds", 
#     "fenugreek", "asafoetida", "curry leaves", "garam masala", "chaat masala", "amchur", 
#     "saunf", "ajwain", "kalonji", "panch phoron", "ghee", "jaggery", "tamarind", 
#     "coconut", "basmati", "besan", "dal", "chana", "moong", "urad", "toor", "masoor", 
#     "rajma", "paneer", "yogurt", "curd", "chilli", "ginger", "garlic"
# ]

# # Regional Indian ingredients to recognize (from your output example)
# REGIONAL_INDIAN_INGREDIENTS = [
#     "badanekayi", "gulkand", "chekke", "amlechi", "kewda", "kuvar", "pulikacha"
# ]

# class IndianRecipeGenerator:
#     def __init__(self, encoder_model, decoder_model, encoder_tokenizer, decoder_tokenizer, 
#                  reverse_decoder_word_index, max_length=100):
#         self.encoder_model = encoder_model
#         self.decoder_model = decoder_model
#         self.encoder_tokenizer = encoder_tokenizer
#         self.decoder_tokenizer = decoder_tokenizer
#         self.reverse_decoder_word_index = reverse_decoder_word_index
#         self.max_length = max_length
        
#         # Special tokens if we need to add them to the tokenizer
#         self.structure_tokens = {
#             "start_ingredients": "<ingredients>",
#             "end_ingredients": "</ingredients>",
#             "start_instructions": "<instructions>",
#             "end_instructions": "</instructions>",
#             "start_recipe": "<start>",
#             "end_recipe": "<end>"
#         }
    
#     def clean_text(self, text):
#         """Clean the generated text by removing non-alphanumeric characters except spaces."""
#         # Remove non-alphanumeric characters except spaces and basic punctuation
#         text = re.sub(r'[^\w\s.,;:!?-]', '', text)
#         # Remove extra spaces
#         text = re.sub(r'\s+', ' ', text)
#         return text.strip()
    
#     def has_proper_sentence_structure(self, text):
#         """Check if the text has proper sentence structure with cooking verbs."""
#         # Split text into sentences
#         sentences = re.split(r'[.!?]+', text)
#         valid_sentences = 0
        
#         for sentence in sentences:
#             sentence = sentence.strip().lower()
#             if not sentence:
#                 continue
                
#             # Check if sentence contains a cooking verb
#             has_verb = any(verb in sentence for verb in INDIAN_COOKING_VERBS)
            
#             # Check if sentence has some basic structure (subject + verb pattern)
#             # This is a simplified check - in reality you'd want more sophisticated NLP
#             if has_verb and len(sentence.split()) >= 3:
#                 valid_sentences += 1
        
#         # Return True if at least 70% of sentences are valid
#         return valid_sentences >= max(1, len(sentences) * 0.7)
    
#     def boost_token_logits(self, logits, current_context=None, instruction_mode=True):
#         """Boost logits for appropriate tokens based on context."""
#         # Get vocabulary as list of words
#         vocab = list(self.reverse_decoder_word_index.values())
        
#         # Determine which kind of tokens to boost based on context
#         boost_tokens = []
#         penalty_tokens = []
        
#         if instruction_mode:
#             # In instruction mode, boost cooking verbs and connectors
#             boost_tokens = INDIAN_COOKING_VERBS + CONNECTOR_WORDS
            
#             # Also boost ingredients but with lower priority
#             boost_tokens += INDIAN_INGREDIENTS + REGIONAL_INDIAN_INGREDIENTS
#         else:
#             # In ingredient mode, boost ingredients
#             boost_tokens = INDIAN_INGREDIENTS + REGIONAL_INDIAN_INGREDIENTS
            
#         # Apply boosts and penalties
#         for i, word in enumerate(vocab):
#             # Apply boost to relevant tokens
#             if any(token in word.lower() for token in boost_tokens):
#                 logits[i] *= 1.2  # 20% boost
                
#             # Penalize repetition if we have context
#             if current_context and word in current_context.split()[-5:]:
#                 logits[i] *= 0.5  # 50% penalty for recently used words
                
#         return logits
    
#     def top_p_filtering(self, logits, top_p=0.92):
#         """Filter logits using nucleus (top-p) sampling."""
#         # Sort logits in descending order
#         sorted_indices = np.argsort(logits)[::-1]
#         sorted_logits = logits[sorted_indices]
        
#         # Calculate cumulative probabilities
#         sorted_probs = np.exp(sorted_logits) / np.sum(np.exp(sorted_logits))
#         cumulative_probs = np.cumsum(sorted_probs)
        
#         # Remove tokens with cumulative probability above threshold
#         sorted_indices_to_remove = cumulative_probs > top_p
#         # Shift indices to keep first token above threshold
#         sorted_indices_to_remove[1:] = sorted_indices_to_remove[:-1]
#         sorted_indices_to_remove[0] = False
        
#         # Create a mask for indices to remove
#         indices_to_remove = np.zeros_like(logits, dtype=bool)
#         indices_to_remove[sorted_indices[sorted_indices_to_remove]] = True
        
#         # Set filtered logits to negative infinity
#         filtered_logits = logits.copy()
#         filtered_logits[indices_to_remove] = -float('inf')
        
#         return filtered_logits
    
#     def generate_recipe(self, input_text, beam_width=5, temperature=1.0, 
#                         top_p=0.92, length_penalty=0.7):
#         """Generate a structured Indian recipe from input text."""
#         # Tokenize input
#         input_seq = self.encoder_tokenizer.texts_to_sequences([input_text])
#         input_seq = np.array(input_seq)
        
#         # Encode input
#         states_value = self.encoder_model.predict(input_seq)
        
#         # Get start and end token indices
#         start_token_index = self.decoder_tokenizer.word_index.get('start', 1)
#         end_token_index = self.decoder_tokenizer.word_index.get('end', 2)
        
#         # Initialize beam search
#         sequences = [([start_token_index], 0.0, states_value, '')]
        
#         # Track all generated n-grams to prevent repetition
#         all_ngrams = defaultdict(int)
        
#         # Flag to track if we're in ingredients or instructions section
#         instruction_mode = False
        
#         # Generate sequence with beam search
#         for _ in range(self.max_length):
#             all_candidates = []
            
#             for seq, score, state, text_so_far in sequences:
#                 if seq[-1] == end_token_index:
#                     all_candidates.append((seq, score, state, text_so_far))
#                     continue
                
#                 # Get last token as input to decoder
#                 target_seq = np.array([[seq[-1]]])
                
#                 # Predict next token
#                 output_tokens, h, c = self.decoder_model.predict([target_seq] + state)
#                 logits = output_tokens[0, -1, :]
                
#                 # Apply temperature
#                 logits = logits / temperature
                
#                 # Switch modes based on special tokens
#                 # This assumes you've added structure tokens to your vocabulary
#                 if "</ingredients>" in text_so_far and not instruction_mode:
#                     instruction_mode = True
                
#                 # Boost appropriate tokens based on mode
#                 logits = self.boost_token_logits(logits, text_so_far, instruction_mode)
                
#                 # Apply top-p filtering
#                 logits = self.top_p_filtering(logits, top_p)
                
#                 # Get top candidates
#                 top_indices = np.argsort(logits)[-beam_width*2:]
                
#                 for idx in top_indices:
#                     if logits[idx] == -float('inf'):
#                         continue  # Skip tokens filtered by top-p
                        
#                     # Calculate probability and log probability
#                     token_prob = np.exp(logits[idx]) / np.sum(np.exp(logits[np.isfinite(logits)]))
#                     token_log_prob = np.log(token_prob + 1e-9)
                    
#                     # Create new candidate
#                     new_seq = seq + [idx]
#                     new_score = score - token_log_prob
#                     new_text = text_so_far
                    
#                     # Convert token to word and add to text
#                     word = self.reverse_decoder_word_index.get(idx, '')
#                     if word and word not in ['start', 'end']:
#                         if new_text and new_text[-1] not in [' ', '\n']:
#                             new_text += ' '
#                         new_text += word
                    
#                     # Check for repetitions
#                     repeat_penalty = 0
                    
#                     # Check for token repetition
#                     if len(new_seq) > 2 and new_seq[-1] == new_seq[-2]:
#                         repeat_penalty += 2.0
                    
#                     # Check for n-gram repetitions
#                     for n in range(2, 5):
#                         if len(new_seq) >= n:
#                             ngram = tuple(new_seq[-n:])
#                             all_ngrams[ngram] += 1
#                             if all_ngrams[ngram] > 1:
#                                 repeat_penalty += 1.0 * all_ngrams[ngram]
                    
#                     # Apply repeat penalty
#                     new_score += repeat_penalty
                    
#                     # Add candidate
#                     all_candidates.append((new_seq, new_score, [h, c], new_text))
            
#             # Apply length normalization
#             normalized_candidates = []
#             for seq, score, state, text in all_candidates:
#                 # Normalize score by length
#                 norm_score = score / (len(seq) ** length_penalty)
#                 normalized_candidates.append((seq, norm_score, state, text))
            
#             # Sort and select top beam_width candidates
#             sequences = sorted(normalized_candidates, key=lambda x: x[1])[:beam_width]
            
#             # Early stopping if all sequences ended
#             if all(seq[-1] == end_token_index for seq, _, _, _ in sequences):
#                 break
        
#         # Get best sequence
#         best_seq, _, _, best_text = sequences[0]
        
#         # Post-process the text
#         recipe = self.post_process_recipe(best_text)
        
#         return recipe
    
#     def post_process_recipe(self, text):
#         """Apply post-processing to the generated recipe text."""
#         # Remove special tokens if present
#         for token in self.structure_tokens.values():
#             text = text.replace(token, '')
        
#         # Clean up the text
#         text = self.clean_text(text)
        
#         # Split into sections based on sentence structure
#         sentences = re.split(r'([.!?] )', text)
#         processed_text = ""
        
#         # Reassemble text with proper sentence structure
#         for i in range(0, len(sentences) - 1, 2):
#             sentence = sentences[i]
#             ending = sentences[i + 1] if i + 1 < len(sentences) else ". "
            
#             # Fix common issues
#             # 1. Ensure sentence starts with capital letter
#             if sentence and not sentence[0].isupper():
#                 sentence = sentence[0].upper() + sentence[1:]
            
#             # 2. Make sure cooking verbs are used properly
#             has_verb = any(verb in sentence.lower() for verb in INDIAN_COOKING_VERBS)
#             if not has_verb and len(sentence.split()) > 3:
#                 # Try to inject a cooking verb if missing
#                 words = sentence.split()
#                 for i, word in enumerate(words):
#                     if word.lower() in INDIAN_INGREDIENTS or word.lower() in REGIONAL_INDIAN_INGREDIENTS:
#                         # Add a cooking verb after an ingredient
#                         verb_index = min(i + 1, len(words))
#                         words.insert(verb_index, "cook")
#                         break
#                 sentence = " ".join(words)
            
#             processed_text += sentence + ending
        
#         # Add the last sentence if there's an odd number
#         if len(sentences) % 2 == 1:
#             last_sentence = sentences[-1]
#             if last_sentence and not last_sentence[0].isupper():
#                 last_sentence = last_sentence[0].upper() + last_sentence[1:]
#             processed_text += last_sentence
            
#             # Make sure it ends with a period
#             if not processed_text.endswith(('.', '!', '?')):
#                 processed_text += '.'
        
#         # Structure the recipe into steps
#         recipe_lines = processed_text.strip().split('. ')
#         structured_recipe = ""
        
#         for i, line in enumerate(recipe_lines):
#             if line:
#                 # Add step numbers
#                 structured_recipe += f"{i+1}. {line}"
#                 # Add period if missing
#                 if not structured_recipe.endswith(('.', '!', '?')):
#                     structured_recipe += '.'
#                 structured_recipe += '\n'
        
#         return structured_recipe.strip()
    
#     def format_with_structure_tokens(self, recipe_text):
#         """Add structure tokens to the recipe if not present."""
#         if "<ingredients>" not in recipe_text:
#             # Try to identify ingredients and instructions sections
#             lines = recipe_text.strip().split('\n')
#             ingredients_section = []
#             instructions_section = []
            
#             in_ingredients = True
            
#             for line in lines:
#                 if line.strip():
#                     if (any(x in line.lower() for x in ['step', 'instruction', 'method', 'procedure']) or 
#                         re.match(r'^\d+\.', line)):
#                         in_ingredients = False
                    
#                     if in_ingredients:
#                         ingredients_section.append(line)
#                     else:
#                         instructions_section.append(line)
            
#             # Format with structure tokens
#             formatted_recipe = "<ingredients>\n"
#             formatted_recipe += '\n'.join(ingredients_section)
#             formatted_recipe += "\n</ingredients>\n\n<instructions>\n"
#             formatted_recipe += '\n'.join(instructions_section)
#             formatted_recipe += "\n</instructions>"
            
#             return formatted_recipe
#         else:
#             return recipe_text

In [66]:
# import pickle
# import numpy as np
# from keras.models import load_model
# from keras.preprocessing.sequence import pad_sequences
# import ipywidgets as widgets
# from IPython.display import display, clear_output

# # Load model
# model = load_model('/kaggle/input/final-food-rep/seq2seq_model.h5')

# # Load tokenizers
# with open('/kaggle/input/final-food-rep/encoder_tokenizer.pkl', 'rb') as f:
#     encoder_tokenizer = pickle.load(f)
# with open('/kaggle/input/final-food-rep/decoder_tokenizer.pkl', 'rb') as f:
#     decoder_tokenizer = pickle.load(f)

# reverse_decoder_word_index = {i: word for word, i in decoder_tokenizer.word_index.items()}
# start_token_index = decoder_tokenizer.word_index['start']
# end_token_index = decoder_tokenizer.word_index['end']

# max_encoder_seq_length = model.input[0].shape[1]
# max_decoder_seq_length = model.output.shape[1]

# # # Generate a recipe
# # recipe = generator.generate_recipe("chicken onion tomato butter")
# # print(recipe)

# # import os
# # os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Only fatal errors


# generator = IndianRecipeGenerator(
#     encoder_model, 
#     decoder_model, 
#     encoder_tokenizer, 
#     decoder_tokenizer, 
#     reverse_decoder_word_index
# )

# # --- UI Components ---
# input_box = widgets.Text(
#     value='onion tomato garlic',
#     placeholder='Enter ingredients...',
#     description='Ingredients:',
#     layout=widgets.Layout(width='600px')
# )
# output_box = widgets.Output()
# button = widgets.Button(description="Generate Instruction", button_style='success')

# def on_button_click(b):
#     output_box.clear_output()
#     input_text = input_box.value.lower()
#     sequence = encoder_tokenizer.texts_to_sequences([input_text])
#     padded_seq = pad_sequences(sequence, maxlen=max_encoder_seq_length, padding='post')
    
#     with output_box:
#         prediction = generate_recipe(padded_seq)
#         print(f"Instruction:\n{prediction}")

# button.on_click(on_button_click)

# # --- Display UI ---
# display(input_box, button, output_box)
